In [ ]:
#%pip install bleak

In [ ]:
import asyncio
from bleak import BleakScanner, BleakClient

In [ ]:
print("Scanning for BLE devices…")
devices = await BleakScanner.discover(timeout=5.0)

if not devices:
    print("No devices found.")
else:
    for d in devices:
        print(f"Name: {d.name!r}, Address: {d.address}")


In [ ]:
TARGET_NAME = "Arduino"   # Change to your peripheral’s advertised name
CHAR_UUID   = "19B10001-E8F2-537E-4F6C-D104768A1214"

def read_flatworm_direction() -> int:
    # replace this stub with real signal‑processing logic.
    # For now it always returns 0.
    return 0

async def main():
    client = None
    try:
        while True:
            # --- (re)connect if needed ---
            if client is None or not client.is_connected:
                print("Scanning for", TARGET_NAME, "…")
                devices = await BleakScanner.discover(timeout=5.0)
                addr = next((d.address for d in devices if d.name == TARGET_NAME), None)
                if addr is None:
                    print(f"{TARGET_NAME} not found; retrying in 2 s…")
                    await asyncio.sleep(2.0)
                    continue

                client = BleakClient(addr)
                try:
                    await client.connect()
                    print("Connected to", TARGET_NAME)
                except Exception as e:
                    print("Connection failed:", e)
                    client = None
                    await asyncio.sleep(2.0)
                    continue

            # --- once connected, read & send the stubbed value ---
            direction = read_flatworm_direction()
            if 0 <= direction <= 6:
                try:
                    await client.write_gatt_char(CHAR_UUID, bytes([direction]))
                    print(f"→ Sent: {direction}")
                except Exception as e:
                    print("Write failed:", e)

            # --- handle disconnection ---
            if client and not client.is_connected:
                print("Disconnected; will reconnect…")
                await client.disconnect()
                client = None

            # small delay before next iteration (2 seconds before next command)
            await asyncio.sleep(2)
    except KeyboardInterrupt:
        if client and client.is_connected:
            await client.disconnect()
        print("Stopped by user.")

# Run in Jupyter’s event loop:
await main()

